In [1]:
import google.generativeai as genai
import json
import os
import time
from pathlib import Path
from tqdm import tqdm

In [2]:
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# See https://ai.google.dev/api/python/google/generativeai/GenerativeModel
generation_config = {
  "temperature": 0.5,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "application/json",
}
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE",
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE",
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE",
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE",
  },
]

In [3]:
data_dir = "data/eutils_raw/"
api_docs = []

for _file in Path(data_dir).iterdir():
    with open(_file) as f:
        doc = f.readlines()
    doc = "".join(doc)
    api_docs.append(doc)

In [4]:
with open("data/prompts/apidoc2json.md") as f:
    base_prompt = f.readlines()
base_prompt = "".join(base_prompt)

In [5]:

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash-latest",
  safety_settings=safety_settings,
  generation_config=generation_config,
  system_instruction="You are a helpful, meticulous, scientific programming and research assistant.",
)


responses = []
for doc in tqdm(api_docs):
  prompt = base_prompt.replace("{API_DOC}", doc)

  response = model.generate_content(prompt)
  try:
    text = json.loads(response.text)
    responses.append(text)
    time.sleep(0.5)
  except ValueError:
    try: 
      print(response.candidates)
    except:
      print("Gemini is throwing a tantrum.")

  0%|          | 0/9 [00:00<?, ?it/s]


InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting